In [7]:
import csv
from collections import Counter
import pandas as pd
from math import log
from sklearn.feature_extraction.text import TfidfVectorizer
import json

In [2]:
year_list = ['2011', '2012', '2013','2014','2015','2016','2017','2018','2019','2020']

In [5]:
for year in year_list:
    word_list = list()
    tmp_line = list()
    file = open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/'+year+'.csv','r',encoding='utf-8')
    reader = csv.reader(file)
    
    for line in reader:
        tmp_line = [word.rstrip('\n') for word in line if word != '']
        word_list = word_list + tmp_line
        
    #word_list = [word.rstrip('\n') for word in word_list]
    
    counter = Counter(word_list)
    word_count = dict(counter.most_common())
    
    file.close()
    
    with open ('C:/Users/happy/Coding/jupyter/crawler/MROpaper/frequency/fre/'+year+'.json','w') as f:
        json.dump(word_count, f)

KeyboardInterrupt: 

In [8]:
# 2011~2020 빈도수 총합
total = dict()
for year in year_list:
    tmp_line = list()
    file = open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/frequency/fre/'+year+'.json','r',encoding='utf-8')
    tmp = json.load(file)
    
    total = Counter(total) + Counter(tmp)
    #word_list = [word.rstrip('\n') for word in word_list]
    
    file.close()

counter = Counter(total)
word_count = dict(counter.most_common())

with open ('C:/Users/happy/Coding/jupyter/crawler/MROpaper/frequency/fre/2011_2020.json','w') as f:
    json.dump(word_count, f)

In [11]:
file = open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/frequency/fre/fre_2011_2020.json','r',encoding='utf-8')
fre = json.load(file)
file.close()

with open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/frequency/fre/fre_2011_2020.csv','w',newline = '',encoding='utf-8') as f:  
    writer = csv.writer(f)
    for k, v in fre.items():
        writer.writerow([k, v])

In [3]:
#tf-idf
for year in year_list:
    doc_list = list()
    word_list = list()
    doc_cnt = 0
    txt = ''

    file = open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/'+year+'.csv','r',encoding='utf-8')
    reader = csv.reader(file)

    for line in reader:
        word_list = [word.rstrip('\n') for word in line]
        txt = ' '.join(word_list)    
        doc_list.append(txt)
        doc_cnt += 1
    
    file.close()

    vectorizer  = TfidfVectorizer(min_df = 2, max_features = 500, analyzer = 'word')

    tfidfv = vectorizer.fit(doc_list)
    tfidfa = tfidfv.transform(doc_list).toarray()
    
    df = pd.DataFrame(tfidfa, columns = sorted(tfidfv.vocabulary_))

    df.to_csv('C:/Users/happy/Coding/jupyter/crawler/MROpaper/frequency/tf-idf/'+year+'.csv',mode = 'w')

In [ ]:
# tf-idf 2011~2020 총합
doc_list = list()
doc_cnt = 0

for year in year_list:
    abstract = list()
    txt = ''

    file = open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/'+year+'.csv','r',encoding='utf-8')
    reader = csv.reader(file)

    for line in reader:
        abstract = [word.rstrip('\n') for word in line]
        txt = ' '.join(abstract)    
        doc_list.append(txt)
        doc_cnt += 1
    
    file.close()

vectorizer  = TfidfVectorizer(min_df = 2, max_features = 500, analyzer = 'word')

tfidfv = vectorizer.fit(doc_list)
tfidfa = tfidfv.transform(doc_list).toarray()

df = pd.DataFrame(tfidfa, columns = sorted(tfidfv.vocabulary_))

df.to_csv('C:/Users/happy/Coding/jupyter/crawler/MROpaper/frequency/tf-idf/2011_2020.csv',mode = 'w')

In [ ]:
---------------------------테스트--------------------------------------

In [42]:
doc_list = list()
word_list = list()
doc_cnt = 0
txt = ''

file = open('C:/Users/happy/Coding/jupyter/crawler/MROpaper/2011.csv','r',encoding='utf-8')
reader = csv.reader(file)

for line in reader:
    word_list = [word.rstrip('\n') for word in line]
    txt = ' '.join(word_list)    
    doc_list.append(txt)
    doc_cnt += 1

file.close()

vectorizer  = TfidfVectorizer(min_df = 2, max_features = 500, analyzer = 'word')

tfidfv = vectorizer.fit(doc_list)
tfidfa = tfidfv.transform(doc_list)

In [40]:
import gensim
from sklearn.decomposition import LatentDirichletAllocation

In [43]:
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

lda_top = lda_model.fit_transform(tfidfa)

terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: [('use', 72.04), ('system', 67.15), ('study', 60.92), ('model', 58.42), ('paper', 57.33)]
Topic 2: [('use', 1.2), ('phase', 1.07), ('increase', 1.07), ('space', 0.97), ('time', 0.88)]
Topic 3: [('model', 5.93), ('paper', 5.67), ('use', 5.55), ('system', 5.44), ('safety', 5.16)]
Topic 4: [('crack', 8.2), ('repair', 6.07), ('composite', 4.93), ('stress', 4.89), ('structure', 4.27)]
Topic 5: [('lean', 1.07), ('production', 0.79), ('state', 0.78), ('design', 0.78), ('approach', 0.76)]
Topic 6: [('aircraft', 0.99), ('safety', 0.76), ('regulation', 0.74), ('price', 0.65), ('change', 0.65)]
Topic 7: [('energy', 0.88), ('model', 0.59), ('work', 0.59), ('country', 0.54), ('well', 0.52)]
Topic 8: [('system', 1.35), ('use', 1.14), ('industry', 1.02), ('model', 1.01), ('leadership', 0.93)]
Topic 9: [('infrastructure', 0.67), ('resource', 0.64), ('life', 0.51), ('new', 0.51), ('system', 0.51)]
Topic 10: [('aircraft', 1.57), ('impact', 1.38), ('failure', 1.37), ('use', 1.3), ('maintenance',

In [36]:
df = pd.DataFrame(tfidfa, columns = sorted(tfidfv.vocabulary_))

df.to_csv('tfidf_2011.csv',mode = 'w')